In [195]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [196]:
from bs4 import BeautifulSoup
import bs4
from datetime import datetime

In [197]:
pip install mailslurp-client


Note: you may need to restart the kernel to use updated packages.


In [198]:
import mailslurp_client

configuration = mailslurp_client.Configuration()
configuration.api_key['x-api-key'] = "a61e1a5227cee7d8c1e3bf759ad2e43330a7883565415d6ba668ec6bbe2e314d"

In [199]:
import pprint
import mailslurp_client
from mailslurp_client.rest import ApiException

In [200]:
pp = pprint.PrettyPrinter(indent=4)

In [201]:
# email_id = '9ac55027-b2f1-43af-bf8e-6c36a7888294'
email_id= '67fd1c5b-53ab-4840-804d-95ab2d8df5a1'#'d10fa4d3-880b-46b1-9c30-2ba829c19d4e'#'8a079ecb-9d84-4045-801e-350a3ed8df5f'

In [202]:
with mailslurp_client.ApiClient(configuration) as api_client:
    api_instance = mailslurp_client.EmailControllerApi(api_client)
    decode = False
    try:
        api_response = api_instance.get_email_html(email_id, decode=decode)
#         pp.pprint(api_response)
    except ApiException as e:
        print("Exception when calling EmailControllerApi->get_email_html: %s\n" % e)

In [203]:
pip install lxml


Note: you may need to restart the kernel to use updated packages.


In [204]:
soup = BeautifulSoup(api_response, 'lxml')

In [205]:
soup.find_all('h4')

[<h4>********* INDEX ***********</h4>,
 <h4>Biotech and Healthcare</h4>,
 <h4>Business and Finance </h4>,
 <h4>Education </h4>,
 <h4>General</h4>,
 <h4>Lifestyle and Fitness</h4>,
 <h4>Public Policy and Government</h4>,
 <h4>****************************</h4>,
 <h4><strong>Biotech and Healthcare</strong></h4>,
 <h4><strong>Business and Finance </strong></h4>,
 <h4><strong>Education </strong></h4>,
 <h4><strong>General</strong></h4>,
 <h4><strong>Lifestyle and Fitness</strong></h4>,
 <h4><strong>Public Policy and Government</strong></h4>]

In [206]:
categories = soup.find_all('h4')

In [207]:
all_links = {}
for h in categories:
    a = h.find_all('a')
    all_links[h.text] = a
# html = u""
# for tag in soup.find("h4").next_siblings:
#     if tag.name == "h4":
#         break
#     else:
#         html += str(tag)

In [208]:
categories

[<h4>********* INDEX ***********</h4>,
 <h4>Biotech and Healthcare</h4>,
 <h4>Business and Finance </h4>,
 <h4>Education </h4>,
 <h4>General</h4>,
 <h4>Lifestyle and Fitness</h4>,
 <h4>Public Policy and Government</h4>,
 <h4>****************************</h4>,
 <h4><strong>Biotech and Healthcare</strong></h4>,
 <h4><strong>Business and Finance </strong></h4>,
 <h4><strong>Education </strong></h4>,
 <h4><strong>General</strong></h4>,
 <h4><strong>Lifestyle and Fitness</strong></h4>,
 <h4><strong>Public Policy and Government</strong></h4>]

In [209]:
for e in soup.findAll('br'):
    e.extract()

In [210]:
reporters_in_category = {}
for category in categories:
    all_tags = []
    for tag in category.next_siblings:
        if tag.name == "h4":
            break
        else:
            all_tags.append(tag)
    reporters_in_category[category.text] = all_tags

In [211]:
reporters_in_category

{'********* INDEX ***********': ['\n',
  <p>
  <a href="javascript:void(0)" id="BiotechandHealthcare" name="BiotechandHealthcare"></a></p>],
 'Biotech and Healthcare': ['\r\n1)',
  <a id="aten" name="aten"></a>,
  " Summary: When is it Safe to Return to Doctor's/Dentists/Mental Health Care\r\n",
  '\n',
  <p>Name: Kerri  Fivecoat-Campbell
  Leading Online Site for People Over 50
  </p>,
  'Category: Biotech and Healthcare\r\n',
  '\r\nEmail: ',
  <a href="mailto:query-aten@helpareporter.net">query-aten@helpareporter.net</a>,
  '\n',
  '\r\nMedia Outlet: Leading Online Site for People Over 50',
  '\r\nDeadline: 7:00 PM EST - 22 May\r\n',
  '\n',
  <p>
  Query: 
  
  Studies have shown that many people have delayed non-essentialappointments to the doctor, dentist and even for mental healthcare during the pandemic. Many of our readers are over 60 andhave underlying health conditions that makes them vulnerable. Ineed to speak with healthcare professionals about when it's safeto return to m

In [212]:
stories = []
story = None
final_category = None
for category in reporters_in_category:
    if "***" not in category:
        final_category = category
        for each_block in reporters_in_category[category]:
#             if isinstance(each_block, bs4.element.Tag):
#                 print(":::::::::::::::::::::::::::::::")
#                 print(each_block)
            if isinstance(each_block, bs4.element.NavigableString):
                if "Summary" in each_block.string:
                    summary = each_block.string.split("Summary: ")[1]
                    if story is not None:
                        story["category"] = category
                        stories.append(story)
                        story = None
                    story = {"summary": summary}
                if "Media Outlet" in each_block.string:
                    media_outlet = each_block.string.split("Media Outlet: ")[1]
                    story["media_outlet"] = media_outlet   
                if "Deadline" in each_block.string:
                    deadline = each_block.string.split("Deadline: ")[1].split("-")
                    deadline_date = deadline[1]
                    deadline_time = deadline[0]
                    story["deadline_date"] = deadline_date
                    story["deadline_time"] = deadline_time
                if "12051 Indian Creek Ct., Beltsville, MD 20705, USA" in each_block.string:
                    break
            if isinstance(each_block, bs4.element.Tag):
                if each_block.name == 'p':
                    if "Name: " in each_block.text:
                        story["name"] = each_block.text.split("Name: ")[1] 
                    if "Query specifics:" in each_block.text:
                        story["query_specifics"] = each_block.text.split("Query specifics: ")[1] 
                    if "Query:" in each_block.text:
                        story["query"] = each_block.text.split("Query: ")[1] 
                    if "Requirements" in each_block.text:
                        story["requirements"] = each_block.text.split("Requirements: ")[1]
                if each_block.name == 'a':
                    if "target" in each_block.attrs:
                        story["email"] = each_block.attrs['href']
story["category"] = final_category
if story is not None:
    stories.append(story)

In [213]:
len(stories)

29

In [214]:
pip install airtable-python-wrapper

Note: you may need to restart the kernel to use updated packages.


In [215]:
from airtable import Airtable
import os
os.environ["AIRTABLE_API_KEY"] = "keytaFveWwlPvHFjw"
airtable = Airtable('app26blbVX3rcmY5p', 'leads')

In [216]:
airtable.get_all()

[{'id': 'rec056MGygkr0SCZv',
  'fields': {'id': 196},
  'createdTime': '2020-05-21T23:03:12.000Z'},
 {'id': 'rec0H8OSeOCnmPnCL',
  'fields': {'id': 48},
  'createdTime': '2020-05-21T22:58:39.000Z'},
 {'id': 'rec0OYNK4gEyyIzo9',
  'fields': {'id': 249},
  'createdTime': '2020-05-22T00:11:51.000Z'},
 {'id': 'rec0eEmKlom8H6QzP',
  'fields': {'id': 257},
  'createdTime': '2020-05-22T00:11:52.000Z'},
 {'id': 'rec0qD3zvG50sxe4q',
  'fields': {'id': 110},
  'createdTime': '2020-05-21T23:01:36.000Z'},
 {'id': 'rec1QexAziSdhCNKW',
  'fields': {'id': 56},
  'createdTime': '2020-05-21T22:59:57.000Z'},
 {'id': 'rec1c5FG0olAoyzYb',
  'fields': {'id': 92},
  'createdTime': '2020-05-21T23:01:33.000Z'},
 {'id': 'rec1sezD40vstbdIT',
  'fields': {'id': 175},
  'createdTime': '2020-05-21T23:03:08.000Z'},
 {'id': 'rec2CdKzGS7KdCw7y',
  'fields': {'id': 285,
   'category': 'Business and Finance ',
   'summary': 'Looking for Marketing Experts for tips on improving Ecommerce Conversion Rate\r\n',
   'media_o

In [217]:
for story in stories:
    airtable.insert(story)
print("Done")

Done
